In [62]:
import matplotlib.pyplot as plt
import networkx as nx
from random import randint
from ipywidgets import interact
from IPython.display import display
import ipywidgets as widgets
import subprocess
%matplotlib inline

In [16]:
G = nx.Graph()
with open('graph.txt', 'r') as f:
    n, m = map(int, f.readline().strip().split())
    G.add_nodes_from(range(n))
    for _ in range(m):
        u, v = map(int, f.readline().strip().split())
        G.add_edge(u, v)

In [130]:
W, H = 100, 100
layouts = [{i: (randint(0, W), randint(0, H)) for i in G} for _ in range(1)]

In [131]:
nodesW = widgets.FloatLogSlider(description='Nodes', min=0, max=3, value=1)
borderW = widgets.FloatLogSlider(description='Border', min=0, max=3, value=1)
edgesW = widgets.FloatLogSlider(description='Edges', min=-3, max=0, value=0.01)
crossW = widgets.FloatLogSlider(description='Crossings')
T = widgets.FloatLogSlider(description='Temperature', min=0, max=5, value=1e4)
cooling = widgets.Text(description='Cooling', value='0.90')
iters = widgets.IntSlider(description='Iterations', min=1, max=10000, value = 1000)
button = widgets.Button(description='Launch annealing')

def launch_annealing(button):
    try:
        float(cooling.value)
    except:
        print('Cooling temperature (%s) is incorrect' % cooling.value)
        return
    with open('start.txt', 'w') as f:
        f.write('\n'.join(map(lambda x: "%s %s" % (x[0], x[1]), (layouts[-1][i] for i in sorted(list(G.nodes))))))
    new_info = subprocess.run([
        "./main",
        "--nodes", str(nodesW.value),
        "--edges", str(edgesW.value),
        "--border", str(borderW.value),
        "--cross", str(crossW.value),
        "--startT", str(T.value),
        "--cooling", str(cooling.value),
        "--rounds", str(iters.value),
        "--width", str(W),
        "--height", str(H),
        "--starting-layout", "start.txt"
    ], capture_output=True).stdout.decode('utf-8').split('\n')
    for line in new_info:
        coords = dict(enumerate(map(lambda x: tuple(map(float, x.split(' '))), line.split(';')[:-4])))
        layouts.append(coords)
    print('Added %d new layouts, now history contains %d layouts' % (len(new_info), len(layouts)))

def do_reset(button):
    W, H = 100, 100
    global layouts
    layouts = layouts[:1]
    print('Resetting: now have %d layouts' % len(layouts))
    
display(nodesW)
display(borderW)
display(edgesW)
display(crossW)
display(T)
display(cooling)
display(iters)
display(button)
button.on_click(launch_annealing)
reset = widgets.Button(description='Reset')
display(reset)
reset.on_click(do_reset)

FloatLogSlider(value=1.0, description='Nodes', max=3.0)

FloatLogSlider(value=1.0, description='Border', max=3.0)

FloatLogSlider(value=0.01, description='Edges', max=0.0, min=-3.0)

FloatLogSlider(value=1.0, description='Crossings')

FloatLogSlider(value=10000.0, description='Temperature', max=5.0)

...

IntSlider(value=1000, description='Iterations', max=10000, min=1)

Button(description='Launch annealing', style=ButtonStyle())

Button(description='Reset', style=ButtonStyle())

Added 10002 new layouts, now history contains 10003 layouts


KeyError: 0

In [132]:
def draw(t):
    nx.draw(G, pos=layouts[t], with_labels=True)
    plt.xlim(0, W)
    plt.ylim(0, H)
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.axhline(H, color='black')
    plt.axvline(W, color='black')
    plt.show()
it = interact(draw, t=widgets.IntSlider(min=0,max=len(layouts)-1,step=1,value=0))

interactive(children=(IntSlider(value=0, description='t', max=10002), Output()), _dom_classes=('widget-interac…